In [1]:
import cvxpy as cp
import numpy as np

In [2]:
#Problem Input
Qx = 0.08
delta = 0.005

# #test1
# K = 1 # Define the number of elements in p and m
# p = np.array([0.03]) # Example values
# m = np.array([1])  # Example values, ensure sum(m) = 1

#test2
np.random.seed(123)
K = 5 # Define the number of elements in p and m
p = np.random.uniform(0, Qx, K)
# Generate random values for m such that all elements add up to 1
m = np.random.dirichlet(np.ones(K))
print("p:", p)
print("m:", m)

assert len(p) == K
assert len(m) == K
assert np.sum(m) == 1

p: [0.05571753 0.02289115 0.01814812 0.04410518 0.05755752]
m: [0.08078721 0.5802404  0.16957052 0.09629893 0.07310294]


In [3]:
# Define the optimization variables
w = cp.Variable()
omega = cp.Variable(K)

In [4]:
# Define the constraints
constraints = [
    omega >= 0,
    omega <= 1,
    w == cp.sum(omega),
    Qx - cp.sum(m * p) - w + 2 * cp.sum(cp.multiply(m, cp.multiply(omega, p))) <= delta,
    Qx - cp.sum(m * p) - w + 2 * cp.sum(cp.multiply(m, cp.multiply(omega, p))) >= -delta
]

In [5]:
# Define the objective function
objective = cp.Maximize(w)

In [6]:
# Formulate the problem
prob = cp.Problem(objective, constraints)

In [7]:
# Solve the problem
prob.solve()

# Print the results
if prob.status == cp.OPTIMAL or prob.status == cp.OPTIMAL_INACCURATE:
    print("Optimal value of w:", w.value)
    print("Optimal values of omega:", omega.value)
else:
    print("Something went wrong!")
    print(prob.status)

Optimal value of w: 0.05720367724929709
Optimal values of omega: [9.80331630e-10 5.72036717e-02 2.08808633e-09 1.20579510e-09
 1.23999238e-09]


In [8]:
if K==1:
    assert(np.isclose(w.value, (Qx-np.sum(p)+delta)/(1-2*np.sum(p))))
elif K>1:
    assert(w.value <= Qx)